Подготовка данных

In [37]:
import copy
from collections import Counter

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\n', '').lower().replace('ё', 'е').replace('ъ', 'ь').replace('—', '-x')
    chars = Counter(text)
    chars = dict(sorted(chars.items(), key=lambda item: item[1], reverse=True))
all_sum = 0
for key, value in chars.items():
    all_sum += value
    print(f'Count of {key} is {value}')
print()
with open('output.txt', 'w', encoding='utf-8') as f:
    for key, value in chars.items():
        f.write(f'{key},{value}\n')
chars = {key: round(value / all_sum, 4) for key, value in chars.items()}
for key, value in chars.items():
    print(f'Probability of {key} is {value}')

Count of   is 64
Count of е is 54
Count of т is 41
Count of о is 37
Count of а is 29
Count of н is 28
Count of с is 27
Count of в is 22
Count of и is 21
Count of к is 20
Count of р is 17
Count of л is 17
Count of м is 17
Count of д is 14
Count of у is 11
Count of з is 11
Count of , is 9
Count of ч is 8
Count of п is 8
Count of ь is 7
Count of г is 6
Count of ы is 6
Count of я is 6
Count of б is 5
Count of ш is 4
Count of ю is 4
Count of х is 4
Count of … is 4
Count of ! is 3
Count of ц is 3
Count of й is 3
Count of ; is 2
Count of - is 1
Count of x is 1
Count of щ is 1

Probability of   is 0.1243
Probability of е is 0.1049
Probability of т is 0.0796
Probability of о is 0.0718
Probability of а is 0.0563
Probability of н is 0.0544
Probability of с is 0.0524
Probability of в is 0.0427
Probability of и is 0.0408
Probability of к is 0.0388
Probability of р is 0.033
Probability of л is 0.033
Probability of м is 0.033
Probability of д is 0.0272
Probability of у is 0.0214
Probability of з is 0

Создание кодов

In [38]:
def huffman(chars):
    if len(chars) == 2:
        return dict(zip(chars.keys(), ['0', '1']))

    new_chars = chars.copy()
    a1, a2 = lowest_prob_pair(chars)
    p1, p2 = new_chars.pop(a1), new_chars.pop(a2)
    new_key = (a1, a2)
    new_chars[new_key] = p1 + p2

    c = huffman(new_chars)
    c_a1_a2 = c.pop(new_key)
    c[a1] = c_a1_a2 + '0'
    c[a2] = c_a1_a2 + '1'

    return c


def lowest_prob_pair(p):
    sorted_p = sorted(p.items(), key=lambda x: x[1])
    return sorted_p[0][0], sorted_p[1][0]


codes = huffman(chars)
for key, value in codes.items():
    print(f'{key}: {value}')

 : 100
е: 001
о: 1100
т: 1101
а: 0110
н: 0101
с: 0001
в: 11110
к: 11100
и: 11101
м: 10110
р: 10100
л: 10101
д: 01001
з: 00000
у: 111111
,: 101111
п: 011110
ч: 011101
я: 010000
ь: 010001
г: 000010
ы: 000011
б: 1111100
…: 1011100
ю: 0111110
х: 0111111
й: 0111000
ш: 0111001
!: 11111010
ц: 11111011
;: 10111010
щ: 101110110
-: 1011101110
x: 1011101111


Показатели эффективности

In [39]:
import math

q_med = 0
H = 0
for key, value in chars.items():
    q_med += round(value * len(codes[key]), 4)
    H += round(-1 * value * math.log(value, 2), 4)
q_med = round(q_med, 4)
print(f'qср = {q_med}')
print(f'H = {H}')
print(f'Показатель эффективности равен {q_med - H}')

qср = 4.5127
H = 4.487599999999998
Показатель эффективности равен 0.0251000000000019


Рисование графа

In [40]:
from anytree import Node, RenderTree
from anytree.exporter import DotExporter

cur_code = ''
all_nodes = {}


def make_tree(chars: dict):
    global cur_code
    root = Node('Main')
    all_nodes['Main'] = root
    if len(chars.items()) == 2:
        buf = list(chars.items())
        if buf[0][1] == '0':
            Node(buf[0][0], parent=root)
            Node(buf[1][0], parent=root)
        else:
            Node(buf[1][0], parent=root)
            Node(buf[0][0], parent=root)
    else:
        cur_code = '0'
        make_main_nodes({key: value for key, value in chars.items() if value[0] == cur_code}, root)
        cur_code = '1'
        make_main_nodes({key: value for key, value in chars.items() if value[0] == cur_code}, root)
    for pre, fill, node in RenderTree(root):  # Render
        print(f'{pre}{node.name}')
    DotExporter(root).to_picture("Haffman.png")


def make_main_nodes(chars: dict, root):
    global cur_code
    if len(chars.items()) > 1:
        all_nodes[cur_code] = Node(cur_code, parent=root)
    new_chars = dict(sorted(chars.items(), key=lambda x: len(x[1])))
    for item in new_chars.items():
        code = item[1]
        if len(code) == 1:
            Node(item, parent=root)
            break
        check_node(code[:len(code) - 1])
        Node(item, parent=all_nodes[code[:len(code) - 1]])


def check_node(node):
    if not node in all_nodes.keys():
        check_node(node[:len(node) - 1])
        if len(node) > 1:
            all_nodes[node] = Node(node, parent=all_nodes[node[:len(node) - 1]])


make_tree(codes)

Main
├── 0
│   ├── 00
│   │   ├── ('е', '001')
│   │   └── 000
│   │       ├── ('с', '0001')
│   │       └── 0000
│   │           ├── ('з', '00000')
│   │           └── 00001
│   │               ├── ('г', '000010')
│   │               └── ('ы', '000011')
│   └── 01
│       ├── 011
│       │   ├── ('а', '0110')
│       │   └── 0111
│       │       ├── 01111
│       │       │   ├── ('п', '011110')
│       │       │   └── 011111
│       │       │       ├── ('ю', '0111110')
│       │       │       └── ('х', '0111111')
│       │       └── 01110
│       │           ├── ('ч', '011101')
│       │           └── 011100
│       │               ├── ('й', '0111000')
│       │               └── ('ш', '0111001')
│       └── 010
│           ├── ('н', '0101')
│           └── 0100
│               ├── ('д', '01001')
│               └── 01000
│                   ├── ('я', '010000')
│                   └── ('ь', '010001')
└── 1
    ├── 10
    │   ├── (' ', '100')
    │   └── 101
    │       ├── 1011
    │ 